In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import (
    impute,
    pipeline,
    preprocessing,
    feature_selection,
    ensemble
)
from pathlib import Path
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.metrics import make_scorer, r2_score


In [ ]:
# Imposta i tuoi percorsi
iacv_path = 'My Drive/AML/prj1'
env_path = Path('/content/drive') / iacv_path
data_path = env_path / 'Data'

In [ ]:
# Carica il training data
train_df_X = pd.read_csv(data_path / 'X_train.csv', skiprows=1, header=None)

# Labels training Data
train_df_Y = pd.read_csv(data_path / 'y_train.csv', skiprows=1, header=None)

# Carica il test data
test_df = pd.read_csv(data_path / 'X_test.csv' , skiprows=1, header=None)

# Split features and target
X_train = train_df_X.values[:, 1:]
y_train = train_df_Y.values[:, 1:]
x_test = test_df.values[:, 1:]


print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape}")


X_train shape: (1212, 832)
y_train shape: (1212, 1)
x_test shape: (776, 832)


DATA CLEAN preso da dani

In [ ]:

class CollinearityFilter(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.95, method='pearson'):
        self.threshold = threshold
        self.method = method
        self.to_drop_ = []
    def fit(self, X, y=None):
        X_df = pd.DataFrame(X)
        corr_matrix = X_df.corr(method=self.method).abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        self.to_drop_ = [column for column in upper.columns if any(upper[column] > self.threshold)]
        return self
    def transform(self, X):
        X_df = pd.DataFrame(X)
        return X_df.drop(columns=self.to_drop_).to_numpy()

class TopKCorrelatedFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, k=100):
        self.k = k
        self.selected_features_ = []
    def fit(self, X, y):
        X_df = pd.DataFrame(X)
        y_series = pd.Series(y.squeeze())
        corrs = {}
        for col in X_df.columns:
            corrs[col] = abs(X_df[col].corr(y_series))
        corrs_series = pd.Series(corrs).fillna(0)
        k_effettivo = min(self.k, len(corrs_series))
        self.selected_features_ = corrs_series.nlargest(k_effettivo).index.tolist()
        return self
    def transform(self, X):
        X_df = pd.DataFrame(X)
        return X_df[self.selected_features_].to_numpy()

def preprocess_data_exact_v2(X_train_raw, y_train_raw, X_test_raw):
    """
    Pipeline ridefinita con i parametri della tabella per XGBoost.
    """
    print("--- Avvio Pipeline 'clean_data_v2' (Versione XGB) ---")
    y_train_flat = y_train_raw.ravel()
    steps = []

    # 1. Collinearity filter (threshold=0.98)
    print("Step 1: Aggiunta CollinearityFilter (threshold=0.98)")
    steps.append((
        'collinearity_filter_pearson',
        CollinearityFilter(threshold=0.98, method='pearson')
    ))

    # 3. Imputer (imputer_type='simple-mean')
    print("Step 3: Aggiunta SimpleImputer (strategy=mean)")
    steps.append((
        'imputer',
        impute.SimpleImputer(strategy='mean')
    ))

    # 4. Scaler (scaler_type='standard')
    print("Step 4: Aggiunta StandardScaler")
    steps.append((
        'scaler',
        preprocessing.StandardScaler()
    ))

    # 5. Variance treshold (variance_threshold=0.0)
    print("Step 5: Aggiunta VarianceThreshold (threshold=0.0)")
    steps.append((
        'variance_threshold',
        feature_selection.VarianceThreshold(threshold=0.0)
    ))

    # --- Creazione ed Esecuzione Pipeline ---
    print("\nCreazione dell'oggetto sklearn.pipeline.Pipeline...")
    pipeline_obj = pipeline.Pipeline(steps)

    print("Avvio pipeline.fit_transform(X_train)...")
    X_train_processed = pipeline_obj.fit_transform(X_train_raw, y_train_flat)

    print("Avvio pipeline.transform(X_test)...")
    X_test_processed = pipeline_obj.transform(X_test_raw)
    print(f"Pipeline completata. Shape dati processati: {X_train_processed.shape}")

    # 6. Anomaly Detection (isoforest-100-0.05)
    print("\n--- Step 6: Avvio Anomaly Detection (IsolationForest) ---")
    iso_forest = ensemble.IsolationForest(
        n_estimators=100,
        contamination=0.05,
        random_state=42
    )
    train_preds = iso_forest.fit_predict(X_train_processed)
    inlier_mask = (train_preds == 1)
    X_train_final = X_train_processed[inlier_mask]
    y_train_final = y_train_flat[inlier_mask]

    print(f"Campioni originali: {X_train_processed.shape[0]}, Campioni rimossi: {np.sum(train_preds == -1)}, Campioni finali: {X_train_final.shape[0]}")
    print("\n--- Preprocessing Pipeline (Versione XGB) Terminato ---")

    return X_train_final, y_train_final, X_test_processed

MODEL DEFINITION

In [ ]:
# --- CELLA 1.5: DEFINIZIONE MODELLO XGBOOST E VALUTAZIONE ---

# 1. Definizione dello "Starting Model" (parametri dalla tabella)
xgb_starting_model = XGBRegressor(
    colsample_bytree=0.6,
    gamma=0.5333,
    learning_rate=0.02,
    max_depth=7,
    min_child_weight=15,
    n_estimators=600,
    random_state=42,
    reg_lambda=5.0,
    subsample=0.7,
    n_jobs=-1
)

# 2. Nuova funzione di valutazione (per XGBoost)
def train_and_evaluate_xgb(X, y, model, n_splits=10):
    """
    Valuta un modello XGBoost (o qualsiasi modello)
    usando la Cross-Validation.
    """
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=cv, scoring=make_scorer(r2_score), n_jobs=-1)
    return scores.mean()

print("Definiti 'xgb_starting_model' e 'train_and_evaluate_xgb'.")

Definiti 'xgb_starting_model' e 'train_and_evaluate_xgb'.


**CLEAN** **DATA**

In [ ]:
# (Assicurati che X_train, y_train, x_test siano caricate dalla cella precedente)

print("Esecuzione del preprocessing...")

# Esegui la funzione di pipeline
X_processed, y_processed, X_test_processed = preprocess_data_exact_v2(X_train, y_train, x_test)

print("\n--- Shape Finali ---")
print(f"X_train_processed shape: {X_processed.shape}")
print(f"y_train_processed shape: {y_processed.shape}")
print(f"X_test_processed shape: {X_test_processed.shape}")


Esecuzione del preprocessing...
--- Avvio Pipeline 'clean_data_v2' (Versione XGB) ---
Step 1: Aggiunta CollinearityFilter (threshold=0.98)
Step 2: Aggiunta TopKCorrelatedFeatures (k=200)
Step 3: Aggiunta SimpleImputer (strategy=mean)
Step 4: Aggiunta StandardScaler
Step 5: Aggiunta VarianceThreshold (threshold=0.0)

Creazione dell'oggetto sklearn.pipeline.Pipeline...
Avvio pipeline.fit_transform(X_train)...


/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Avvio pipeline.transform(X_test)...
Pipeline completata. Shape dati processati: (1212, 140)

--- Step 6: Avvio Anomaly Detection (IsolationForest) ---
Campioni originali: 1212, Campioni rimossi: 61, Campioni finali: 1151

--- Preprocessing Pipeline (Versione XGB) Terminato ---

--- Shape Finali ---
X_train_processed shape: (1151, 140)
y_train_processed shape: (1151,)
X_test_processed shape: (776, 140)


FEATURE SELECTION

In [ ]:
#BACKWARDS FEATURE ELIMINATION FOR FEATURE SELECTION

import xgboost as xgb
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector as SFS

sfs = SFS(
    estimator=xgb_starting_model,
    n_features_to_select='auto',
    tol=0.001,
    direction='backward',
    scoring='r2',
    cv=3,
)
sfs = sfs.fit(X_processed, y_processed)
X_processed = sfs.transform(X_processed)
X_test_processed = sfs.transform(X_test_processed)




MODEL EVALUATION

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import warnings

# Assicurati che X_processed e y_processed siano definiti dalle celle precedenti

# 1. Definisci lo spazio di ricerca (Search Space)
# Usiamo i tuoi parametri di xgb_starting_model come centro
space = {
    # hp.quniform(label, low, high, q) -> restituisce round(uniform(low, high) / q) * q
    'max_depth': hp.quniform('max_depth', 5, 10, 1),  # Intorno a 7 (valori interi 5, 6, 7, 8, 9, 10)
    'min_child_weight': hp.quniform('min_child_weight', 10, 20, 1), # Intorno a 15

    # hp.uniform(label, low, high) -> restituisce un valore uniforme tra low e high
    'gamma': hp.uniform('gamma', 0.1, 1.0), # Intorno a 0.53
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.8), # Intorno a 0.6
    'reg_lambda': hp.uniform('reg_lambda', 2.0, 8.0) # Intorno a 5.0
}

# 2. Definisci la funzione obiettivo (Objective Function)
# Questa è la funzione che hyperopt cercherà di minimizzare
def objective(params):
    """
    Funzione obiettivo da minimizzare (in questo caso, -R^2).
    Utilizza un K-Fold manuale per implementare correttamente l'early stopping.
    """

    # Converte i parametri float (come max_depth) in interi dove necessario
    params['max_depth'] = int(params['max_depth'])
    params['min_child_weight'] = int(params['min_child_weight'])

    # Parametri fissi (presi dal tuo modello e dalle best practices)
    fixed_params = {
        'learning_rate': 0.02,
        'n_estimators': 2000,       # Alto, per early stopping
        'subsample': 0.7,
        'random_state': 42,
        'tree_method': 'auto',      # Usa la GPU se disponibile
        'n_jobs': -1,
        'early_stopping_rounds': 50,
        'eval_metric': 'rmse'
    }

    # Combina i parametri dello spazio di ricerca con quelli fissi
    model_params = {**params, **fixed_params}

    # Imposta la cross-validation (3 fold per velocità)
    kf = KFold(n_splits=3, shuffle=True, random_state=42)

    scores = []

    # Loop di K-Fold manuale
    for train_index, val_index in kf.split(X_processed, y_processed):
        X_train, X_val = X_processed[train_index], X_processed[val_index]
        y_train, y_val = y_processed[train_index], y_processed[val_index]

        # Inizializza il modello CON i parametri di early stopping
        model = xgb.XGBRegressor(**model_params)

        # Addestra il modello passando l'eval_set
        # Questo è il modo corretto che evita tutti gli errori precedenti
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_val, y_val)],
            verbose=False # Silenzia l'output di addestramento
        )

        # Calcola e salva lo score
        preds = model.predict(X_val)
        fold_score = r2_score(y_val, preds)
        scores.append(fold_score)

    # Calcola la media degli score
    mean_r2_score = np.mean(scores)

    # hyperopt minimizza, quindi restituiamo il R^2 negativo
    return {'loss': -mean_r2_score, 'status': STATUS_OK}

# 3. Esegui l'ottimizzazione
print("--- Avvio Ottimizzazione Bayesiana (hyperopt) ---")

# Trials() salva la storia della ricerca
trials = Trials()

best_params = fmin(
    fn=objective,      # Funzione obiettivo
    space=space,         # Spazio di ricerca
    algo=tpe.suggest,    # Algoritmo (Tree-structured Parzen Estimator)
    max_evals=10,        # Numero di combinazioni da provare (es. 50-100)
    trials=trials,       # Oggetto per salvare i risultati
    rstate=np.random.default_rng(42) # Per riproducibilità
)

print("\n--- Ottimizzazione Completata ---")
print("Migliori parametri trovati (valori grezzi):")
print(best_params)

# Converte i valori 'best' in interi
best_params_clean = best_params.copy()
best_params_clean['max_depth'] = int(best_params_clean['max_depth'])
best_params_clean['min_child_weight'] = int(best_params_clean['min_child_weight'])

print("\nMigliori parametri (pronti all'uso):")
print(best_params_clean)

# Calcola il punteggio R^2 migliore
best_r2_score = -min(trials.losses())
print(f"\nMiglior punteggio R² (CV medio): {best_r2_score:.5f}")

--- Avvio Ottimizzazione Bayesiana (hyperopt) ---
  0%|          | 0/10 [00:08<?, ?trial/s, best loss=?]


KeyboardInterrupt: 

In [ ]:
print("\n--- Esecuzione Predizione Finale e Sottomissione ---")
# Crea e addestra il modello finale sull'intero set di training processato
final_model  = xgb.XGBRegressor(**best_params_clean)
final_model.fit(X_processed, y_processed)

# Predici sul set di test processato (X_test_selected)
y_test_pred = final_model.predict(X_test_processed)

print("Predizioni generate.")

# --- Crea il file di sottomissione ---
# Formato richiesto: 'id' e 'y'
submission_df = pd.DataFrame({
    'id': np.arange(0, len(y_test_pred)),
    'y': y_test_pred.flatten()
})

# Salva il file nel tuo percorso Google Drive
submission_path = env_path / 'submission.csv'
submission_df.to_csv(submission_path, index=False)

print(f"File di sottomissione salvato in: {submission_path}")
display(submission_df.head())


--- Esecuzione Predizione Finale e Sottomissione ---
Predizioni generate.
File di sottomissione salvato in: /content/drive/My Drive/AML/prj1/submission.csv


,id,y
0,0,61.322960
1,1,78.042191
2,2,73.886497
3,3,74.328094
4,4,72.235031
